In [1]:

import os, time, copy, math, random, json
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, WeightedRandomSampler
import torchvision
from torchvision import transforms, datasets, models
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score
from tqdm.auto import tqdm

# ------------------ Configuration ------------------
DATA_PATH = Path('data_ss')
TRAIN_DIR, VAL_DIR, TEST_DIR = DATA_PATH/'train', DATA_PATH/'val', DATA_PATH/'test'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
USE_AMP = True if DEVICE.type == 'cuda' else False

BATCH_SIZE = 32
IMAGE_SIZE = 224
EPOCHS = 50
LR = 3e-4
WEIGHT_DECAY = 1e-4
PATIENCE = 8
NUM_WORKERS = 4
SEED = 42

SAVE_BEST = 'best_mobilenetv2_mose.pth'
TTA = 4

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if DEVICE.type == 'cuda': torch.cuda.manual_seed_all(SEED)


c:\Users\user\Desktop\MoSE\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


*SE block for extra attention layer in MobileNetV2 base model*

In [ ]:

# Squeeze and Excitation block (SE)
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super().__init__()
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channel, channel // reduction, 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel // reduction, channel, 1),
            nn.Sigmoid()
        )
    def forward(self, x): return x * self.fc(x)


*getting base model MobileNetV2, changing last layer -> adding SE block*

In [ ]:

# MobileNetV2-SE block
def get_model_mobilenet_v2(num_classes, pretrained=True, se=True):
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT if pretrained else None)
    if se:
        last_conv_ch = model.features[-1][0].out_channels if hasattr(model.features[-1][0], 'out_channels') else 1280
        model.features.add_module('se_block', SEBlock(last_conv_ch))
    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_features, num_classes))
    return model


*data augmentation for better model training, tuning*

In [ ]:

# Data Augmentation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.3, 0.3, 0.3, 0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE*1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
val_dataset   = datasets.ImageFolder(VAL_DIR, transform=val_test_transforms)
test_dataset  = datasets.ImageFolder(TEST_DIR, transform=val_test_transforms)

NUM_CLASSES = len(train_dataset.classes)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)


*train, test, validation step*

In [ ]:

# train step
from torch.cuda.amp import autocast, GradScaler

def train_step(model, dataloader, loss_fn, optimizer, device):
    model.train()
    scaler = GradScaler(enabled=USE_AMP)
    metric_acc = MulticlassAccuracy(num_classes=NUM_CLASSES, average='macro').to(device)
    total_loss = 0

    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        with autocast(enabled=USE_AMP):
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        metric_acc.update(y_pred.argmax(1), y)

    return total_loss / len(dataloader), metric_acc.compute().item()

# validation, test step
def val_or_test_step(model, dataloader, loss_fn, device, mode='val'):
    model.eval()
    total_loss = 0
    average_type = 'weighted'
    acc = MulticlassAccuracy(num_classes=NUM_CLASSES, average=average_type).to(device)
    prec = MulticlassPrecision(num_classes=NUM_CLASSES, average=average_type).to(device)
    rec = MulticlassRecall(num_classes=NUM_CLASSES, average=average_type).to(device)
    f1 = MulticlassF1Score(num_classes=NUM_CLASSES, average=average_type).to(device)

    with torch.inference_mode():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            with autocast(enabled=USE_AMP):
                y_pred = model(X)
                loss = loss_fn(y_pred, y)
            total_loss += loss.item()
            preds = y_pred.argmax(1)
            acc.update(preds, y); prec.update(preds, y); rec.update(preds, y); f1.update(preds, y)

    metrics = {
        f'{mode}_loss': total_loss / len(dataloader),
        f'{mode}_acc': acc.compute().item(),
        f'{mode}_prec': prec.compute().item(),
        f'{mode}_rec': rec.compute().item(),
        f'{mode}_f1': f1.compute().item()
    }
    return metrics


*model training loop*

In [ ]:

# model training
def train_model(model, train_loader, val_loader, test_loader, loss_fn, optimizer, epochs, device):
    best_val_acc = 0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'test_acc': [], 'test_prec': [], 'test_rec': [], 'test_f1': []}

    for epoch in tqdm(range(epochs), desc='Training Epochs'):
        train_loss, train_acc = train_step(model, train_loader, loss_fn, optimizer, device)
        val_metrics = val_or_test_step(model, val_loader, loss_fn, device, mode='val')
        test_metrics = val_or_test_step(model, test_loader, loss_fn, device, mode='test')

        print(f"""\nEpoch [{epoch+1}/{epochs}] 
        Train: Loss={train_loss:.4f} | Acc={train_acc:.4f}
        Val:   Loss={val_metrics['val_loss']:.4f} | Acc={val_metrics['val_acc']:.4f}
        Test:  Acc={test_metrics['test_acc']:.4f} | Prec={test_metrics['test_prec']:.4f} | Rec={test_metrics['test_rec']:.4f} | F1={test_metrics['test_f1']:.4f}""")

        if val_metrics['val_acc'] > best_val_acc:
            best_val_acc = val_metrics['val_acc']
            torch.save(model.state_dict(), SAVE_BEST)
            print(f"Saved new best model ({best_val_acc:.4f})")

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_metrics['val_loss'])
        history['val_acc'].append(val_metrics['val_acc'])
        history['test_acc'].append(test_metrics['test_acc'])
        history['test_prec'].append(test_metrics['test_prec'])
        history['test_rec'].append(test_metrics['test_rec'])
        history['test_f1'].append(test_metrics['test_f1'])

    return history


In [ ]:

# training command
model = get_model_mobilenet_v2(num_classes=NUM_CLASSES, pretrained=True, se=True).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

results = train_model(model, train_loader, val_loader, test_loader, loss_fn, optimizer, EPOCHS, DEVICE)

with open('mobilenetv2_mose_results.json', 'w') as f:
    json.dump(results, f)

print("\nTraining complete. Results saved to mobilenetv2_mose_results.json")


Training Epochs:   0%|          | 0/50 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_3844\3836392956.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=USE_AMP)
C:\Users\user\AppData\Local\Temp\ipykernel_3844\3836392956.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=USE_AMP):
C:\Users\user\AppData\Local\Temp\ipykernel_3844\3836392956.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=USE_AMP):
Training Epochs:   2%|▏         | 1/50 [01:43<1:24:26, 103.40s/it]


Epoch [1/50] 
        Train: Loss=0.9710 | Acc=0.7172
        Val:   Loss=0.2967 | Acc=0.8994
        Test:  Acc=0.8766 | Prec=0.8870 | Rec=0.8766 | F1=0.8687
✅ Saved new best model (0.8994)


Training Epochs:   4%|▍         | 2/50 [03:03<1:11:57, 89.95s/it] 


Epoch [2/50] 
        Train: Loss=0.2673 | Acc=0.9093
        Val:   Loss=0.1872 | Acc=0.9254
        Test:  Acc=0.9107 | Prec=0.9207 | Rec=0.9107 | F1=0.9083
✅ Saved new best model (0.9254)


Training Epochs:   6%|▌         | 3/50 [04:24<1:06:59, 85.53s/it]

...


Epoch [48/50] 
        Train: Loss=0.0486 | Acc=0.9711
        Val:   Loss=0.0721 | Acc=0.9621
        Test:  Acc=0.9636 | Prec=0.9646 | Rec=0.9636 | F1=0.9631


Training Epochs:  98%|█████████▊| 49/50 [1:05:53<01:19, 79.62s/it]


Epoch [49/50] 
        Train: Loss=0.0566 | Acc=0.9665
        Val:   Loss=0.0866 | Acc=0.9609
        Test:  Acc=0.9659 | Prec=0.9673 | Rec=0.9659 | F1=0.9658


Training Epochs: 100%|██████████| 50/50 [1:07:14<00:00, 80.69s/it]


Epoch [50/50] 
        Train: Loss=0.0496 | Acc=0.9746
        Val:   Loss=0.0810 | Acc=0.9680
        Test:  Acc=0.9659 | Prec=0.9684 | Rec=0.9659 | F1=0.9642

✅ Training complete. Results saved to mobilenetv2_mose_results.json
